In [1]:
import cv2 as cv
import numpy as np
import operator
import pickle
import os

In [4]:
# 定义画图函数
def cv_show(name, img):
    cv.imshow(name, img)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [3]:
# 读取从视频中截取的图像
image = cv.imread(r'D:\opencv-ziliao\park\test_images\scene1410.jpg')
cv_show('image', image)

In [4]:
#掩膜以去除背景

# 制作mask
lower = np.uint8([120, 120, 120])
upper = np.uint8([255, 255, 255])
# lower_red和高于upper_red的部分分别变成0，lower_red～upper_red之间的值变成255,相当于过滤背景
white_mask = cv.inRange(image, lower, upper)
cv_show('white_mask',white_mask)

In [5]:
# 与操作去除背景
white_yellow_image = cv.bitwise_and(image, image, mask = white_mask)
cv_show('white_yellow_image', white_yellow_image)

In [6]:
# 转灰度图
gray_image = cv.cvtColor(white_yellow_image, cv.COLOR_BGR2GRAY)
cv_show('gray_image', gray_image)

In [7]:
# 边缘检测
edge_image = cv.Canny(gray_image, 50, 200)
cv_show('edge_image', edge_image)

In [8]:
# 手动选择停车场区域
# 在停车场区域周围做几个标记来将此区域围起来
row, col = image.shape[:2]
pt_1 = [col*0.05, row*0.90]
pt_2 = [col*0.05, row*0.70]
pt_3 = [col*0.30, row*0.55]
pt_4 = [col*0.6, row*0.15]
pt_5 = [col*0.90, row*0.15] 
pt_6 = [col*0.90, row*0.90]

vertices = np.array([[pt_1, pt_2, pt_3, pt_4, pt_5, pt_6]], dtype=np.int32)
point_img = edge_image.copy()
point_img = cv.cvtColor(point_img, cv.COLOR_GRAY2RGB)
for point in vertices[0]:
    cv.circle(point_img, (point[0], point[1]), 10, (0, 0, 255), 4)
cv_show('point_img', point_img)

In [9]:
# 将这几个红色圆圈所包围的地方做一个mask
mask = np.zeros_like(edge_image)
cv.fillPoly(mask, vertices, 255)
cv_show('mask', mask)

In [10]:
# 过滤操作，删除居民楼等区域
roi_image = cv.bitwise_and(edge_image, mask)
cv_show('roi_image', roi_image)

In [11]:
# 霍夫变换检测直线
lines = cv.HoughLinesP(roi_image, rho=0.1, theta=np.pi/10, threshold=15, minLineLength=9, maxLineGap=4)
print(lines.shape)

(2169, 1, 4)


In [12]:
# 挑出符合要求的直线并画出
line_image = np.copy(image)
cleaned = []
for line in lines:
    x1, y1, x2, y2 = line[0]
    if abs(y2-y1) <= 1 and abs(x2-x1) >=25 and abs(x2-x1) <= 55:
        cleaned.append((x1, y1, x2, y2))
        cv.line(line_image, (x1, y1), (x2, y2), [255, 0, 0], 2)
print("No lines detected: ", len(cleaned))
cv_show('line_image', line_image)     

No lines detected:  559


In [13]:
# 按列划分区域

# 将得到的直线cleaned进行排序
# 由于得到的列表中的直线没有顺序，所以要先进行排序，排序原则：x1越来越大，x1相同的情况下，y1越来越大。
rect_image = np.copy(image)
list1 = sorted(cleaned, key=operator.itemgetter(0, 1))

In [14]:
# 将在同一列的直线放到字典中的一个键中

clusters = {}
dIndex = 0
clus_dist = 20
for i in range(len(list1) - 1):
    distance = abs(list1[i+1][0] - list1[i][0])
    if distance <= clus_dist:
        if not dIndex in clusters.keys(): 
            clusters[dIndex] = []
        clusters[dIndex].append(list1[i])
        clusters[dIndex].append(list1[i + 1]) 

    else:
        dIndex += 1
#得到含有键值为0-11的字典clusters。clusters中的每个键代表一列区域中的所有直线的端点坐标。

In [15]:
# 得到矩形坐标
# set()去重。 排序。一列中至少有5条直线
# 纵坐标 = 一列中上下两边界的纵坐标，横坐标 = 列中所有直线两端点坐标平均值

rect_coord = {}
i = 0
for key in clusters:
    all_list = clusters[key]
    list2 = list(set(all_list))
    if len(list2) > 5:
        list2 = sorted(list2, key=lambda tup: tup[1])
        avg_y1 = list2[0][1]
        avg_y2 = list2[-1][1]
        avg_x1 = 0
        avg_x2 = 0
        for tup in list2:
            avg_x1 += tup[0]
            avg_x2 += tup[2]
        avg_x1 = avg_x1/len(list2)
        avg_x2 = avg_x2/len(list2)
        rect_coord[i] = (avg_x1, avg_y1, avg_x2, avg_y2)
        i += 1
print("Num Parking Lanes: ", len(rect_coord))


Num Parking Lanes:  12


In [16]:
buff = 7  # 微调
for key in rect_coord:
    tup_topLeft = (int(rect_coord[key][0] - buff), int(rect_coord[key][1]))
    tup_botRight = (int(rect_coord[key][2] + buff), int(rect_coord[key][3]))
    cv.rectangle(rect_image, tup_topLeft,tup_botRight,(0,255,0),3)
cv_show('rect_image', rect_image)


In [17]:
# 在每个列区域画出横线
delineated = np.copy(image)
gap = 15.5  # 同一列中相邻停车位之间的纵向距离
spot_pos = {}
tot_spots = 0
#微调
adj_y1 = {0: 20, 1:-10, 2:0, 3:-11, 4:28, 5:5, 6:-15, 7:-15, 8:-10, 9:-30, 10:9, 11:-32}
adj_y2 = {0: 30, 1: 50, 2:15, 3:10, 4:-15, 5:15, 6:15, 7:-20, 8:15, 9:15, 10:0, 11:30}

adj_x1 = {0: -8, 1:-15, 2:-15, 3:-15, 4:-15, 5:-15, 6:-15, 7:-15, 8:-10, 9:-10, 10:-10, 11:0}
adj_x2 = {0: 0, 1: 15, 2:15, 3:15, 4:15, 5:15, 6:15, 7:15, 8:10, 9:10, 10:10, 11:0}


for key in rect_coord:
    tup = rect_coord[key]
    x1 = int(tup[0] + adj_x1[key])
    x2 = int(tup[2] + adj_x2[key])
    y1 = int(tup[1] + adj_y1[key])
    y2 = int(tup[3] + adj_y2[key])
    cv.rectangle(delineated, (x1, y1), (x2, y2), (0, 255, 0), 2)
    num_splits = int(abs(y2-y1) // gap)
    for i in range(num_splits + 1):
        y = int(y1 + i * gap)
        cv.line(delineated, (x1, y), (x2, y), [255, 0, 0], 2)
    if key > 0 and key < len(rect_coord) - 1:
        x = int((x1 + x2) / 2)
        cv.line(delineated, (x, y1), (x, y2), [255, 0, 0], 2)
    if key == 0 or key == (len(rect_coord) - 1):
        tot_spots += num_splits + 1
    else:
        tot_spots += 2 * (num_splits + 1)
    if key == 0 or key == (len(rect_coord) - 1):
        for i in range(num_splits + 1):
            cur_len = len(spot_pos)
            y = int(y1 + i * gap)
            spot_pos[(x1, y, x2, y+gap)] = cur_len + 1
    else:
        for i in range(num_splits + 1):
            cur_len = len(spot_pos)
            y = int(y1 + i * gap)
            x = int((x1 + x2) / 2)
            spot_pos[(x1, y, x, y+gap)] = cur_len +1
            spot_pos[(x, y, x2, y+gap)] = cur_len +2 
print("total parking spaces: ", tot_spots, cur_len)
cv_show('delineated', delineated)


total parking spaces:  561 560


In [18]:
# 将得到的每个停车位的位置信息写入文件
with open('spot_dict.pickle', 'wb') as handle:
    pickle.dump(spot_pos, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [19]:
#将得到的每个停车位裁剪下来做成样本
for spot in spot_pos:
    (x1, y1, x2, y2) = spot
    (x1, y1, x2, y2) = (int(x1), int(y1), int(x2), int(y2))
    #裁剪
    spot_img = image[y1:y2, x1:x2]
    spot_img = cv.resize(spot_img, (0,0), fx=2.0, fy=2.0) 
    spot_id = spot_pos[spot]
    filename = 'spot' + str(spot_id) +'.jpg'
#     print(spot_img.shape, filename, (x1,x2,y1,y2))
    cv.imwrite(os.path.join('D:\Jupyter\dataset\cnn_data', filename), spot_img)


In [20]:
import tensorflow as tf
import glob

In [21]:
model = tf.keras.models.load_model('car1.h5')

In [35]:
images = glob.glob('D:\Jupyter\dataset\cnn_data\*.jpg')

In [36]:
images.sort(key=lambda a:int(a.split('\\')[-1].split('t')[1].split('.')[0]))

In [37]:
image_dataset = tf.data.Dataset.from_tensor_slices(images)

In [33]:
def load_image(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [48, 48])
    img = tf.cast(img, tf.float32)
#     img = img/255
    return img

In [38]:
image_dataset = image_dataset.map(load_image)

In [39]:
image_dataset = image_dataset.batch(1)

In [28]:
with open('D:\Jupyter\spot_dict.pickle', 'rb') as f:
    spot_pos = pickle.load(f)

In [29]:
class_dictionary = {}
class_dictionary[0] = 'empty'
class_dictionary[1] = 'occupied'
predicted_images = np.copy(image)
overlay = np.copy(image)


In [40]:
for i,img in enumerate(image_dataset):
    class_predicted = model.predict(img)
    inID = np.argmax(class_predicted[0])
    label = class_dictionary[inID]
    if label == 'empty':
        for key in spot_pos:
            if spot_pos[key] == i+1:
                (x1, y1, x2, y2) = key
                cv.rectangle(overlay, (int(x1),int(y1)), (int(x2),int(y2)), [0, 255, 0], -1)

                
                

In [41]:
alpha = 0.5
predicted_images = cv.addWeighted(overlay, alpha, predicted_images, 1 - alpha, 0, predicted_images)      

In [42]:
cv_show('predicted_images', predicted_images)

In [45]:
# 对录像视频进行测试，实时识别出空停车位位置及数量

video_name = 'D:\opencv-ziliao\park\parking_video.mp4'
cap = cv.VideoCapture(video_name)
count = 0
while 1:
    ret, image = cap.read()
    count += 1
    if count == 5:
        count = 0
        
        new_image = np.copy(image)
        overlay = np.copy(image)
        cnt_empty = 0
        all_spots = 0
        color = [0, 255, 0]
        alpha = 0.5
        for i,img in enumerate(image_dataset):
            class_predicted = model.predict(img)
            inID = np.argmax(class_predicted[0])
            label = class_dictionary[inID]
            if label == 'empty':
                for key in spot_pos:
                    if spot_pos[key] == i+1:
                        (x1, y1, x2, y2) = key
                        cv.rectangle(overlay, (int(x1),int(y1)), (int(x2),int(y2)), color, -1)
                        cnt_empty += 1
        cv.addWeighted(overlay, alpha, new_image, 1 - alpha, 0, new_image)
        cv.putText(new_image, "Available: %d spots" %cnt_empty, (30, 95),
        cv.FONT_HERSHEY_SIMPLEX,
        0.7, (255, 255, 255), 2)

        cv.putText(new_image, "Total: %d spots" %all_spots, (30, 125),
        cv.FONT_HERSHEY_SIMPLEX,
        0.7, (255, 255, 255), 2)
        cv.imshow('frame', new_image)
        if cv.waitKey(10) & 0xFF == ord('q'):
            break
            
cv.destroyAllWindows()
cap.release()

In [6]:
x = [[281,271,57,7],[374,270,65,65]]

In [8]:
len(x)

2

In [12]:
int(x[0][0]*0.1)

28

In [13]:
y = [[323,295,58,58]]

In [14]:
len(y)

1

In [15]:
4%2

0

In [16]:
4/2

2.0